In [2]:
#If you’re happy with the dataset, then load it
from datasets import load_dataset
from transformers import AutoTokenizer

from itertools import islice
from typing import Iterable, List, Tuple

import argparse
import numpy as np
import torch
import datasets
from datasets.dataset_dict import DatasetDict

from gensim.models.keyedvectors import KeyedVectors
from torch import nn
import random

https://medium.com/dataseries/k-fold-cross-validation-with-pytorch-and-sklearn-d094aa00105f
Cross validation

https://medium.com/analytics-vidhya/text-summarization-using-nlp-3e85ad0c6349
text summarization

https://medium.com/analytics-vidhya/text-summarization-using-bert-gpt2-xlnet-5ee80608e961
More text summarization


PREPROCESSING DECISIONS:

1. Labels at every tenth percentile.

2. Max input sequence length - both standard BERT and max-doc-length as separate models.

3. New-lines removed.

In [3]:
# Load datasets
tifu_short_raw = load_dataset("reddit_tifu", "short", split="train")
tifu_long_raw = load_dataset("reddit_tifu", "long", split = "train")

Generating train split:   0%|          | 0/79740 [00:00<?, ? examples/s]

Dataset reddit_tifu downloaded and prepared to /home/coder/.cache/huggingface/datasets/reddit_tifu/short/1.1.0/1c73fb08807b54ec26b025829b2a3d90c6f7466dac20801c825571af9514c049. Subsequent calls will reuse this data.


Generating train split:   0%|          | 0/42139 [00:00<?, ? examples/s]

Dataset reddit_tifu downloaded and prepared to /home/coder/.cache/huggingface/datasets/reddit_tifu/long/1.1.0/1c73fb08807b54ec26b025829b2a3d90c6f7466dac20801c825571af9514c049. Subsequent calls will reuse this data.


In [4]:
# Add decimal labels to upvote ratio
def add_columns(example):
    example["upvote_ratio"] = round(example["upvote_ratio"], 1)
    example["labels"] = int(example["upvote_ratio"] * 10)
    return example

tifu_short = tifu_short_raw.map(add_columns)

tifu_long = tifu_long_raw.map(add_columns)

  0%|          | 0/79740 [00:00<?, ?ex/s]

  0%|          | 0/42139 [00:00<?, ?ex/s]

In [ ]:
random.seed(1337)

# Split into train, validation and test.
dict1 = tifu_short.train_test_split(test_size=0.1)

train_and_val = dict1["train"]
test_short = dict1["test"]

dict2 = train_and_val.train_test_split(test_size=0.2)

train_short = dict2["train"]
val_short = dict2["test"]

In [ ]:
# Tokenize
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenization(example):
    return tokenizer(example["documents"], padding = True, truncation= True, return_tensors = "pt")


train_input = train_short.map(tokenization, batched = True)
val_input = val_short.map(tokenization, batched = True)
test_input = test_short.map(tokenization, batched = True)

In [ ]:
train_input.save_to_disk("preprocessed/train_short")
val_input.save_to_disk("preprocessed/val_short")
test_input.save_to_disk("preprocessed/test_short")

In [ ]:
# Let's train a model!

def prepare_batch(token_ids, labels):
    token_id_tensor = torch.LongTensor(np.array(token_ids)) # Turn list of lists into np.array into torch.LongTensor
    labels = torch.LongTensor(np.array(labels))
    output = tuple([token_id_tensor, labels])
    return output

model_name = "bert-base-uncased"

from datasets import load_from_disk

train_cl_short = load_from_disk("../preprocessed/train_short")
val_cl_short = load_from_disk("../preprocessed/val_short")
test_cl_short = load_from_disk("../preprocessed/test_short")

from torch.utils.data import DataLoader

train_cl_short = train_cl_short.remove_columns(['ups', 'num_comments', 'upvote_ratio', 'score', 'documents', 'tldr', 'title'])
train_cl_short.set_format(type="torch", columns=["input_ids", "token_type_ids", "attention_mask", "labels"])

small_train = train_cl_short.select(range(1000))

train_dataloader = torch.utils.data.DataLoader(small_train, batch_size=32)

from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=11)

from torch.optim import AdamW
optimizer = AdamW(model.parameters(), lr=5e-5)



epochs = 2



from train import train_model
train_model(model, train_dataloader, val_cl_short, optimizer, epochs, model_name = "test_")